In [107]:
import pandas as pd
import jieba
import re
from gensim.models import word2vec
import gensim
import logging


In [108]:
# 载入数据，csv文件一般是逗号分隔的数据集
data = pd.read_csv('./online_shopping_10_cats.csv') 

In [109]:
# 提取“水果类”的评论用作本次实验
data_fruit = data[data.cat == '水果']

In [110]:
# 中文字符提取函数，仅提取评论中的中文字符，忽略标点符号以及英文字母、单词等
def extractChinese(s):
    pattern="[\u4e00-\u9fa5]+"#中文正则表达式
    regex = re.compile(pattern) #生成正则对象 
    results = regex.findall(s) #匹配
    return "". join(results)

In [111]:
# 将原始评论（仅含中文字符）进行分词，最终获得一个corpus列表，列表中的每一项是每个评论对应的分词列表
corpus = []
for rev in data_fruit.review:
    rev = extractChinese(rev)
    corpus.append(list(jieba.cut(rev)))

In [112]:
# 训练并保存doc2vec模型
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

model_path = './fruit_review_d2v.model' #模型保存路径
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(corpus)] #为corpus数据集打标签
model = Doc2Vec(documents, vector_size=300, window=2, min_count=1, workers=4)
model.save(model_path)
model_d2v = Doc2Vec.load('./fruit_review_d2v.model')

In [113]:
# 使用训练好的模型将所有的review表示为向量,并从原始数据集中获取评论的标签
vectors = []
for review in corpus:
    vectors.append(model_d2v.infer_vector(review))

In [114]:
from sklearn.model_selection import train_test_split
x = vectors # 转化为向量后的数据集
y = list(data_fruit.label) #数据集的标签（好评1/差评0）
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=1) #将预处理后的数据集分割为训练集和测试集

In [115]:
#  建立和训练模型、使用模型并计算模型预测准确率
from sklearn.linear_model import LogisticRegression
import numpy as np
LRmodel = LogisticRegression() #建立logistic回归模型
LRmodel.fit(x_train,y_train) # 使用训练集训练模型
y_hat = LRmodel.predict(x_test) # 使用测试集进行类别预测
result = y_hat == y_test # 记录测试集中预测准确的数量
acc = np.mean(result) # 计算模型预测准确率
print(acc)

0.858


In [116]:
comments = pd.read_csv('./comments.txt',header=None)
len(comments.values)

9960

In [125]:
comments_vectors = []
for comment in comments.values:
    comment = extractChinese(comment[0])
    v = model_d2v.infer_vector(list(jieba.cut(comment)))
    comments_vectors.append(v)

In [126]:
print(len(comments_vectors))
types = LRmodel.predict(comments_vectors)
print(len(types))

9960
9960


In [130]:
types[:100]

array([1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [150]:
types_tmp = []
for t in types:
    if (t == 1):
        types_tmp.append('好评')
    else:
        types_tmp.append('差评')
# types = types_tmp
types_tmp

['好评',
 '好评',
 '好评',
 '好评',
 '差评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '差评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '差评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '差评',
 '好评',
 '好评',
 '差评',
 '好评',
 '好评',
 '差评',
 '差评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '差评',
 '好评',
 '差评',
 '差评',
 '好评',
 '差评',
 '好评',
 '好评',
 '差评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '差评',
 '好评',
 '好评',
 '好评',
 '差评',
 '差评',
 '好评',
 '好评',
 '差评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '差评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '差评',
 '好评',
 '好评',
 '好评',
 '差评',
 '差评',
 '好评',
 '好评',
 '好评',
 '差评',
 '差评',
 '好评',
 '好评',
 '好评',
 '差评',
 '好评',
 '好评',
 '好评',
 '好评',
 '差评',
 '差评',
 '好评',
 '好评',
 '好评',
 '差评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '差评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',
 '好评',

In [159]:
comments_type = pd.DataFrame(columns=['comment', 'type'])
# commentsssss = pd.Series(comments.values)
print(type(comments.values))

comments_type['comment'] = pd.Series(list(comments.values))
comments_type['type'] = pd.Series(types_tmp)

<class 'numpy.ndarray'>


In [160]:
comments_type

,comment,type
0,[看电视，玩游戏，不错的。特别是快递，超五星服务，速度也是杠杠的],好评
1,[刚刚才拿到手 开机运行很流畅 一点卡的感觉的都没有 机身触感也很好],好评
2,[一直喜欢家里手机都是所以买平板果断选择了没有失望非常清晰音质漂亮支持!],好评
3,[整体还算满意吧，毕竟价格在这里，送货速度慢了点，这大黑边不是盖的，机器是真机。此平板也就娱...,好评
4,[平板目前用着还不错，系统流畅，暂时没发现什么问题，自带软件还行不是特别多，就是连张膜都不送...,差评
...,...,...
9955,[一、优点：1、屏幕颜色鲜艳2、声音大，像个塑料扬声器一样，我的手机一响，住我楼下的同事都听...,差评
9956,[优点就不多说了。],差评
9957,[待机时间还可以，正常使用，电话也不多，不经常发短信，能待机三天左右吧。EL960的摄像头在...,好评
9958,"[1.microsoftwindowsmobile2003操作系统,可以下载很多好玩的游戏等...",差评


In [161]:
comments_type.to_excel('评价分类.xlsx',index=False)